<a href="https://colab.research.google.com/github/lamld203844/chat-any/blob/main/chat_any.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# System flow

In [ ]:
!git clone https://github.com/lamld203844/LLMOps.git

import os
HOME = os.getcwd()
print(HOME)

%cd {HOME}/LLMOps/lit-chat_with_code_RAG

Cloning into 'LLMOps'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 46 (delta 15), reused 42 (delta 11), pack-reused 0
Receiving objects: 100% (46/46), 766.79 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content
/content/LLMOps/lit-chat_with_code_RAG


## Load file
- load website

In [ ]:
https://cinnamon.is/en/

In [ ]:
%pip install -q llama-index llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=6f98502f4adc5e50035d357c288d3b25535872cc0e795dcaafe6

## Load embeddings model

In [ ]:
%pwd

'/content/LLMOps/lit-chat_with_code_RAG'

In [ ]:
!pip install -q langchain_community sentence_transformers

from torch import cuda
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

import os
os.environ['HF_HOME'] = 'lit-chat_with_code_RAG/weights'

def load_embedding_model(
    model_name: str = "BAAI/bge-large-en-v1.5",
    device: str = "cuda" if cuda.is_available() else "cpu"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model

embedding_model = load_embedding_model()
embedding = embedding_model.embed_query('Hello, world')
print(embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[0.0570748969912529, 0.002089085290208459, 0.031196100637316704, -0.02522590011358261, 0.011089175008237362, -0.0145485894754529, 0.03243298456072807, 0.02435419335961342, 0.012902908958494663, 0.047478292137384415, -0.02691935934126377, 0.005025328136980534, -0.029651692137122154, -0.00635149423032999, -0.04613000527024269, -0.013684586621820927, -0.021656915545463562, -0.024495387449860573, -0.03326606750488281, 0.020560868084430695, -0.0008339070482179523, 0.022403689101338387, -0.09311306476593018, -0.0007678320980630815, -0.01177266612648964, 0.018722308799624443, -0.002855998231098056, -0.0006272841710597277, 0.031226318329572678, 0.06863529980182648, 0.0010834576096385717, 0.03208589181303978, -0.03522685915231705, -0.049050163477659225, -0.005676851142197847, -0.022463645786046982, 0.04617384448647499, -0.05407458171248436, -0.03859327733516693, -0.02402876876294613, -0.006388620473444462, 0.0031947321258485317, 0.04964834824204445, -0.05397965386509895, -0.054031841456890106, 

## Load llm

In [ ]:
!pip install -q llama-index-llms-gemini
!pip install -q llama-index google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [ ]:
from google.colab import userdata
from llama_index.llms.gemini import Gemini

llm = Gemini(model_name="models/gemini-pro", api_key=userdata.get('GOOGLE_API_KEY'))
resp = llm.complete("Write a poem about a magic backpack")
print(resp)

**The Magic Backpack**

In a realm where dreams take flight,
There's a backpack with powers bright.
Its fabric shimmers, a vibrant hue,
A portal to wonders, both old and new.

With a whisper, it opens wide,
Revealing treasures it cannot hide.
Books of knowledge, maps of distant lands,
Tools for adventure, at your commands.

Its straps adjust, a perfect fit,
As if it knows your every bit.
It carries burdens, light and heavy,
A faithful companion, always ready.

Through time and space, it can transport,
To realms unknown, where wonders retort.
With a leap, you're whisked away,
To worlds where magic holds sway.

It's a sanctuary, a secret place,
Where dreams unfold with boundless grace.
Within its depths, a universe resides,
Where imagination freely abides.

So don this backpack, embrace its might,
And embark on adventures, day and night.
For in its magic, you'll find,
A world of wonder, forever entwined.


# GUI with Streamlit


In [ ]:
!pip install -q streamlit
!npm install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.27s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
%%writefile app.py

import streamlit as st

st.write('Hello, *World!* :sunglasses:')

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

104.154.193.143
npx: installed 22 in 2.266s
your url is: https://witty-olives-hunt.loca.lt
